In [54]:
from g4f.client import Client
from g4f.cookies import set_cookies
from datetime import datetime
import json
import os

In [55]:
res_dir = "res/"
har_dir = "hardir/"
gemini_cookies = f"{har_dir}gemini_cookies.json"
authors_list_filepath = f"{res_dir}gutenberg_author_list"
prompts_filepath = f"{res_dir}prompts"

models = ["gemini-pro", "claude-3-opus", "gpt-4-turbo"]
response_length = 500

Setup necessary cookies

In [56]:
with open(gemini_cookies, "r") as f:
    cookies = json.load(f)
    set_cookies(".google.com", {
        "__Secure-1PSID": cookies["__Secure-1PSID"],
        "__Secure-1PSIDTS": cookies["__Secure-1PSIDTS"],
    })

In [57]:
def get_generated_response(system_spec, prompt, model):
    client = Client()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_spec},
            {"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

In [58]:
def save_data(data):
    model_dir = f"{res_dir}{data['model']}"
    filename = f"{model_dir}/{data['author']}/request{data['id']}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        f.write(json.dumps(data, indent=4))

In [59]:
authors = open(authors_list_filepath, 'r', encoding='utf-8').read().split('\n')
authors[:5]

['Abraham Lincoln',
 'Agatha Christie',
 'Albert Einstein',
 'Aldous Huxley',
 'Alexander Pope']

In [60]:
prompts = open(prompts_filepath, 'r', encoding='utf-8').read().split('\n')
prompts[:5]

['Tell me about your favourite weather.',
 'Tell me how would you explain the look of the sun to the blind person.',
 'Explain me the meaning of life.',
 'Explain me how to create cyan color.',
 'Present me the advantage of the salt over the pepper.']

In [61]:
for model in models[2:]:

    for author in authors[:2]: # only 2 authors for testing purposes
        system_spec = f"Answer in {author}'s writing style. Don't use direct references and citations of {author}. Use {response_length} words. Answer in plain text format."
        id = 0
        
        for prompt in prompts: # only 2 prompts for testing purposes
            print(f"Generating response for {author}'s response using {model} model.")
            response = get_generated_response(system_spec, prompt, model)

            data = {
                "requested_response_length": response_length,
                "response_length": len(response),
                "model": model,
                "created_at": datetime.now().isoformat(),
                "author": author, 
                "system_spec": system_spec,
                'prompt': prompt,
                'response':  response
            }

            save_data(data)
            id += 1

Generating response for Abraham Lincoln's response using gpt-4-turbo model.
Generating response for Abraham Lincoln's response using gpt-4-turbo model.
Generating response for Agatha Christie's response using gpt-4-turbo model.
Generating response for Agatha Christie's response using gpt-4-turbo model.
